In [18]:
import numpy as np
import pandas as pd
from tabulate import tabulate
#1339 filas
df = pd.read_csv('../csvs/insurance.csv')
#.strip() coge todas las columnas en este caso de texto y le quita los espacios en blanco.
df.columns = df.columns.str.strip()
np.random.seed(42)
physical_activity_levels = np.array(["sedentary", "moderate", "active"])
df['physical_activity'] = np.random.choice(physical_activity_levels, size = len(df))

#Eliminamos la columna children dado que carece de relevancia real significativa a la hora de calcular un seguro médico personal.
df = df.drop("children", axis=1)

#Usaremos la columna expenses (gastos médicos en el último año), dividiéndolo en deciles para aplicarle un incremento dependiendo del decil correspondiente para usarlo como precio base
decil = pd.qcut(df['expenses'], q=10, labels=False)
expenses = df['expenses']
incremento = 1 + (decil * 0.10)
precio_base = expenses * incremento
df['precio_final'] = precio_base

def ajustar_precio(fila):
    precio = fila['precio_final']  
    #Edad
    if fila['age'] < 25:
        precio *= 0.90  # 10% de descuento
    elif fila['age'] > 75:
        precio *= 1.25  # 25% de incremento
    elif fila['age'] > 50:
        precio *= 1.10  # 10% de incremento

    #Sexo
    if fila['sex'] == 'female':
        if 18 <= fila['age'] <= 45:
            precio *= 1.07  # 7% de incremento
        elif fila['age'] > 45:
            precio *= 1.02  # 2% de incremento

    #BMI
    if fila['bmi'] < 18.5:
        precio *= 1.02  # 2% de incremento
    elif 25 <= fila['bmi'] < 30:
        precio *= 1.05  # 5% de incremento
    elif fila['bmi'] >= 30:
        precio *= 1.10  # 10% de incremento

    #Smoker
    if fila['smoker'] == 'yes':
        precio *= 1.10  # 10% de incremento

    #Región
    if fila['region'] == 'northeast': # 1% de incremento
        precio *= 1.01
    elif fila['region'] == 'southwest': # 3% de incremento
        precio *= 1.03
    elif fila['region'] == 'southeast': # 2% de incremento
        precio *= 1.02
  
    #Physical activity
    if fila['physical_activity'] == 'sedentary': # 5% de incremento
          precio *= 1.05
    elif fila['physical_activity'] == 'moderate': # 2% de incremento
          precio *= 1.02

  
    return precio

df['insurance_price'] = df.apply(ajustar_precio, axis=1)

correlacion = df['expenses'].corr(df['insurance_price'])
print(correlacion)
print(tabulate(df, headers='keys', tablefmt='fancy_grid'))


0.9930967560812091
╒══════╤═══════╤════════╤═══════╤══════════╤═══════════╤════════════╤═════════════════════╤════════════════╤═══════════════════╕
│      │   age │ sex    │   bmi │ smoker   │ region    │   expenses │ physical_activity   │   precio_final │   insurance_price │
╞══════╪═══════╪════════╪═══════╪══════════╪═══════════╪════════════╪═════════════════════╪════════════════╪═══════════════════╡
│    0 │    19 │ female │  27.9 │ yes      │ southwest │   16884.9  │ active              │       28704.4  │          32884.7  │
├──────┼───────┼────────┼───────┼──────────┼───────────┼────────────┼─────────────────────┼────────────────┼───────────────────┤
│    1 │    18 │ male   │  33.8 │ no       │ southeast │    1725.55 │ sedentary           │        1725.55 │           1829.58 │
├──────┼───────┼────────┼───────┼──────────┼───────────┼────────────┼─────────────────────┼────────────────┼───────────────────┤
│    2 │    28 │ male   │  33   │ no       │ southeast │    4449.46 │ active  